In [ ]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!git checkout lmdb

In [ ]:
!pip -q install keras
!pip -q install colorlog
!pip -q install keras-radam

In [ ]:
import os
os.chdir('/content/Generative_Models/')
print(os.getcwd())

# Dataset loading

In [ ]:
dataset_name='pokemon'

In [ ]:
images_dir = '/content/Generative_Models/data/.pokemon/'
validation_percentage = 30
valid_format = 'png'

In [ ]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [ ]:
!ls -la /content/Generative_Models/data/.pokemon

In [ ]:
image_lists = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose = 1
)

In [ ]:
import numpy as np
import lmdb
import pickle
import math
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img

In [ ]:
from utils.data_and_files.file_utils import get_file_path

In [ ]:
class Pokemon_Image:
    def __init__(self, image, label):
        # Dimensions of image for reconstruction - not really necessary 
        # for this dataset, but some datasets may include images of 
        # varying sizes
        self.channels = image.shape[2]
        self.size = image.shape[:2]

        self.image = image.tobytes()
        self.label = label

    def get_images(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.uint8)
        return images.reshape(-1,*self.size, self.channels)

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.uint8)
        # print(images.shape)
        return images.reshape(np.array(self.size, self.channels))

In [ ]:
def store_many_lmdb(filename, images, labels):
    """ Stores an array of images to LMDB.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    map_size = num_images * images[0].nbytes * 10

    # Create a new LMDB DB for all the images
    #str(lmdb_dir / f"{num_images}_lmdb")
    env = lmdb.open(filename, map_size=map_size, create= True)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        for i in range(num_images):
            # All key-value pairs need to be Strings
            value = Pokemon_Image(images[i], labels[i])
            key = f"{i:08}"
            txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

In [ ]:
import sys
def store_single_lmdb(filename, wrapper , index, num_images):
    """ Stores a wrapper to LMDB.
    """

    map_size = wrapper.get_image().nbytes * num_images + sys.getsizeof(wrapper.label) * 2 * num_images
    env = lmdb.open(filename, map_size=map_size, create= True)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
          # All key-value pairs need to be Strings
          value = wrapper
          key = f"{index:08}"
          txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

Test 1

In [ ]:
!rm -rf /content/LMDB/
!mkdir /content/LMDB/

In [ ]:
lmdb_dir = '/content/LMDB'

category='training'

# print('num_class',num_class)
image_dir=images_dir

target_size=None
color_mode='rgb'
# save_to_dir=None,
save_prefix=''
save_format='png'

def store_lmdb(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    for index, _ in enumerate(image_lists[label_name][category]):
        img_path = get_file_path(image_lists,
                                  label_name,
                                  index,
                                  image_dir,
                                  category)
        img = img_to_array(
                        load_img(
                        img_path,
                        grayscale=color_mode=='grayscale',
                        target_size=target_size
                        )
                    )
        name, _ = os.path.splitext(img_path)
        vid_img_arr = name.split(sep=os.sep)[-1:]
        y = [np.array(vid_img_arr)]
        X = np.array(img)
        # print(X.shape)
        # print(X.shape[2])
        # print(X.shape[:2])
        
        name =  lmdb_dir+os.sep+'_{}'.format(category)
        pokemon_Image = Pokemon_Image(X, y)

        store_single_lmdb(index = index, filename=name, wrapper = pokemon_Image, num_images = num_images)
        

        # if (y is None):
        #       y = [np.array(vid_img_arr)]
        # else:
        #       y = np.append(y, [np.array(vid_img_arr)], 0)
        # if (X is None):
        #       X = [np.array(img)]
        # else:
        #       X = np.append(X, [img], 0)
        # if (index+1) % 200 == 0 and index != 0:
        #     print('Loaded {} 200 pictures'.format((index+1) / 200))
        #     print('Saving images as LMDB file')
        #     store_many_lmdb(lmdb_dir+os.sep+'_{}'.format((index+1) / 200), images = X, labels = y)
        #     y = None
        #     X = None
        # elif len(image_lists[label_name][category])-1 == index:
        #     print('Loaded last {} pictures'.format(math.ceil((index+1) / 200)))
        #     print('Saving images as LMDB file')
        #     store_many_lmdb(lmdb_dir+os.sep+'_{}'.format(float(math.ceil((index+1) / 200))), images = X, labels = y)
        #     y = None
        #     X = None
            


In [ ]:
store_lmdb(image_lists, image_dir)

In [ ]:
# In  case we want each label to be a key then

def store_many_lmdb_labelaskey(filename, images, labels):
    """ Stores an array of images to LMDB.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    map_size = num_images * images[0].nbytes * 10

    # Create a new LMDB DB for all the images
    #str(lmdb_dir / f"{num_images}_lmdb")
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        for i in range(num_images):
            # All key-value pairs need to be Strings
            value = Pokemon_Image(images[i], labels[i])
            key = labels[i]
            txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()


In [ ]:
import math
def read_many_lmdb(filename, num_images):
    """ Reads image from LMDB.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []
    #str(lmdb_dir / f"{num_images}_lmdb")
    for folder_indx in  range(math.ceil(num_images/200)):
      env = lmdb.open(filename+os.sep+'_{}'.format(float(folder_indx+1)), readonly=True)
      print(folder_indx)
      # Start a new read transaction
      with env.begin() as txn:
          # Read all images in one single transaction, with one lock
          # We could split this up into multiple transactions if needed
          if num_images- folder_indx*200 >200:
            max_index = 200 
          else:
            max_index = num_images - folder_indx*200
          
          for image_id in range(0,max_index):
              data = txn.get(f"{image_id:08}".encode("ascii"))
              # Remember that it's a CIFAR_Image object 
              # that is stored as the value
              pokemon_image = pickle.loads(data)
              # Retrieve the relevant bits
              images.append(pokemon_image.get_images())
              labels.append(pokemon_image.label)
      env.close()
    return images, labels

Test 2 Reading data


In [ ]:
X, y = read_many_lmdb(lmdb_dir, 809)

In [ ]:
print()

In [ ]:
def store_lmdb(
             lmdb_name,
             image_lists,
             category, 
             image_dir,
             target_size=None,
             color_mode='rgb',
             save_to_dir=None,
             save_prefix='',
             save_format='png'
):

    classes = list(image_lists.keys())
    num_class = len(classes)

    class2id = dict(zip(classes, range(len(classes))))
    id2class = dict((v, k) for k, v in class2id.items())

    if color_mode not in {'rgb', 'grayscale'}:
        raise ValueError('Invalid color mode:', color_mode, '; expected "rgb" or "grayscale".')

    lmdb_env = lmdb.open(lmdb_name)
    # lmdb_db = lmdb_env.begin(write=True)
    # print(classes)
    with lmdb_env.begin(write=True) as lmdb_db:
      for label_name in classes:
          for j, _ in enumerate(image_lists[label_name][category]):
              img_path = get_file_path(image_lists,
                                        label_name,
                                        j,
                                        image_dir,
                                        category)
              img = img_to_array(
                              load_img(
                              img_path,
                              grayscale=color_mode=='grayscale',
                              target_size=target_size
                              )

                          )
              
              str_id.encode('ascii'), datum.SerializeToString()
              X.tobytes()

              
              lmdb_db.put(label_name.encode(), img, append=True, overwrite=False)      
    lmdb_env.close()

In [ ]:
store_lmdb(lmdb_name='pokemon2', image_lists=imgs_list, category='training', image_dir=images_dir)

In [ ]:
lmdb_env = lmdb.open('pokemon2')
with lmdb_env.begin() as lmdb_txn:
    with lmdb_txn.cursor() as lmdb_cursor:
        for key, value in lmdb_cursor:  
           print (key)
           print (value)

In [ ]:
keys = [key for key, _ in lmdb_db.cursor() ]
values = [value for _, value in lmdb_db.cursor() ]

In [ ]:
print(np.fromstring(values))

In [ ]:
import numpy as np
import lmdb

def write_lmdb(filename):
    print ('Write lmdb')

    lmdb_env = lmdb.open(filename, map_size=int(1e9))

    n_samples= 2
    X= (255*np.random.rand(n_samples,3,4,3)).astype(np.uint8)
    y= np.random.rand(n_samples).astype(np.float32)

    for i in range(n_samples):
        with lmdb_env.begin(write=True) as lmdb_txn:
            lmdb_txn.put('X_'+str(i), X)
            lmdb_txn.put('y_'+str(i), y)

            print ('X:',X)
            print ('y:',y)

def read_lmdb(filename):
    print ('Read lmdb')

    lmdb_env = lmdb.open(filename)
    lmdb_txn = lmdb_env.begin()
    lmdb_cursor = lmdb_txn.cursor()

    n_samples=0
    with lmdb_env.begin() as lmdb_txn:
        with lmdb_txn.cursor() as lmdb_cursor:
            for key, value in lmdb_cursor:  
                print (key)
                if('X' in key):
                    print (np.fromstring(value, dtype=np.uint8))
                if('y' in key):
                    print (np.fromstring(value, dtype=np.float32))

                n_samples=n_samples+1

    print ('n_samples',n_samples)

write_lmdb('temp.db')
read_lmdb('temp.db')